## Given a 365 way scene classifier, obtain logits for 16-way scene category classifier


In [10]:
import os, sys
import torch
import torchvision
import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.insert(0, 'src')
from utils.places365_pred_utils import get_class_category_dict
from utils.utils import ensure_dir, read_json
from utils.model_utils import prepare_device
from datasets.datasets import ImageDataset

In [7]:
# Set variables
places365_dir= os.path.join('data', 'Places365')
index_path = os.path.join(places365_dir, 'places365_val.txt')
image_dir = os.path.join(places365_dir, 'val_256')

train_split = 0.6
seed = 0

data_save_dir = os.path.join('data', 'places365_categories')
data_save_path = os.path.join(data_save_dir, 'places365_imagelabels.pth')

config_path = os.path.join('configs', 'learn_category_classifier_resnet_places365.json')

### Split Places 365 into train-val

In [41]:
# Run setup script if not already
if os.path.exists(data_save_path):
    sys.path.insert(0, 'setup')
    from setup_places365_categories import setup_places365_categories
    setup_places365_categories(
        image_dir=image_dir,
        index_path=index_path,
        train_split=train_split,
        data_save_path=data_save_path,
        seed=seed
    )

Path data/places365_categories/places365_imagelabels.pth exists. Aborting


36500it [00:10, 3325.37it/s]


Saved scene and scene category labels
Split train/val (0.6/0.4)
Saved data to data/places365_categories/places365_imagelabels.pth


### Save features of `val_train` and `val_val` partitions using Resnet model

In [ ]:
config_path = os.path.join('configs', 'save_features_places_resnet_places365val.json')
from save_features import save_features

save_features(config_path)

### Load features and labels for training and validation

In [15]:
data_path = os.path.join('data', 'places365_categories', 'places365_imagelabels.pth')
features_dir = os.path.join('saved', 'Places365_val', '0509_161413')
train_features_path = os.path.join(features_dir, 'val_train_features.pth')
val_features_path = os.path.join(features_dir, 'val_val_features.pth')

# Load labels
data = torch.load(data_path)
train_paths = data['val_train']
val_paths = data['val_val']
category_labels = data['scene_category_labels']
train_category_labels = [category_labels[path] for path in train_paths]
val_category_labels = [category_labels[path] for path in val_paths]

# Load features
train_features_dict = torch.load(train_features_path)
val_features_dict = torch.load(val_features_path)

# Sanity checks for elementwise correspondence
for idx, path in enumerate(train_features_dict['paths']):
    assert path == train_paths[idx]
for idx, path in enumerate(val_features_dict['paths']):
    assert path == val_paths[idx]

train_features = train_features_dict['features']
val_features = val_features_dict['features']

# Length sanity checks
assert len(train_features) == len(train_category_labels)
assert len(val_features) == len(val_category_labels)

print("Loaded {} samples for training and {} samples for validation".format(
    len(train_features), len(val_features)
))

dict_keys(['scene_labels', 'scene_category_labels', 'val_train', 'val_val'])
